### Import des librairies

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from bs4 import BeautifulSoup
import os
import spacy
import re

import chardet
from chardet import detect


### Extraction des données

In [15]:
def detect_file_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = detect(raw_data)
        return result['encoding']

path = "data"
for file in os.listdir(path):
    path_file = os.path.join(path, file)
    if os.path.isfile(path_file):
        encoding = detect_file_encoding(path_file)
        print(f"Encoding for {file}: {encoding}")


Encoding for balzac-comedie_humaine-vol01.htm: utf-8
Encoding for balzac-comedie_humaine-vol02.htm: utf-8
Encoding for balzac-comedie_humaine-vol03.htm: utf-8
Encoding for flaubert-bouvard_pecuchet-pg14157.html: utf-8
Encoding for flaubert-education_sentimentale_01.htm: utf-8
Encoding for flaubert-madame_bovary-pg14155.html: utf-8
Encoding for flaubert-salammbo.htm: utf-8
Encoding for maupassant-fort.htm: utf-8
Encoding for maupassant-pierre_et_jean.htm: utf-8
Encoding for maupassant-une_vie-pg17457.html: utf-8
Encoding for sand-chateau_desertes.htm: utf-8
Encoding for sand-fadette.htm: utf-8
Encoding for sand-lucrezia_floriani.htm: utf-8
Encoding for sand-maitres_sonneurs-pg23582.html: utf-8
Encoding for sand-mare_diable-pg23582.html: utf-8
Encoding for sand-meunier_angibault.htm: utf-8
Encoding for zola-argent-pg6497.html: utf-8
Encoding for zola-assommoir-pg6497.html: utf-8
Encoding for zola-bonheur_dames-pg16852.html: utf-8
Encoding for zola-germinal-pg5711.html: utf-8
Encoding for

In [43]:
# import codecs

# def convert_to_utf8(file_path, original_encoding):
#     with codecs.open(file_path, 'r', original_encoding) as file:
#         content = file.read()

#     with codecs.open(file_path, 'w', 'utf-8') as file:
#         file.write(content)

# path = "data"
# for file in os.listdir(path):
#     path_file = os.path.join(path, file)
#     if os.path.isfile(path_file):
#         encoding = detect_file_encoding(path_file)
#         print(f"Encoding for {file}: {encoding}")

#         if encoding != 'utf-8':
#             convert_to_utf8(path_file, encoding)
#             print(f"Converted {file} to UTF-8")

Encoding for balzac-comedie_humaine-vol01.htm: utf-8
Encoding for balzac-comedie_humaine-vol02.htm: ISO-8859-1
Converted balzac-comedie_humaine-vol02.htm to UTF-8
Encoding for balzac-comedie_humaine-vol03.htm: ISO-8859-1
Converted balzac-comedie_humaine-vol03.htm to UTF-8
Encoding for flaubert-bouvard_pecuchet-pg14157.html: utf-8
Encoding for flaubert-education_sentimentale_01.htm: ISO-8859-1
Converted flaubert-education_sentimentale_01.htm to UTF-8
Encoding for flaubert-madame_bovary-pg14155.html: utf-8
Encoding for flaubert-salammbo.htm: ISO-8859-1
Converted flaubert-salammbo.htm to UTF-8
Encoding for maupassant-fort.htm: ISO-8859-1
Converted maupassant-fort.htm to UTF-8
Encoding for maupassant-pierre_et_jean.htm: ISO-8859-1
Converted maupassant-pierre_et_jean.htm to UTF-8
Encoding for maupassant-une_vie-pg17457.html: utf-8
Encoding for sand-chateau_desertes.htm: ISO-8859-1
Converted sand-chateau_desertes.htm to UTF-8
Encoding for sand-fadette.htm: ISO-8859-1
Converted sand-fadette.h

In [16]:
path = "data"
textes = []
labels = []

for file in os.listdir(path):
    path_file = os.path.join(path, file)

    if os.path.isfile(path_file):
        with open(path_file, encoding='utf-8') as fp:
            soup = BeautifulSoup(fp, "html.parser")

        paragraphes = soup.select("p")
        for p in paragraphes:
            texte = p.get_text()
            textes.append(texte)
            labels.append(file.split("-")[0])

print("Nombre d'échantillons")
print("Balzac :",labels.count("balzac"))
print("Flaubert :",labels.count("flaubert"))
print("Maupassant :",labels.count("maupassant"))
print("Sand :",labels.count("sand"))
print("Zola :",labels.count("zola"))

Nombre d'échantillons
Balzac : 6898
Flaubert : 11028
Maupassant : 6536
Sand : 7309
Zola : 15462


Vérifications

In [18]:
# Afficher les 10 premiers textes prétraités
for i in range(10):
    print(f"Texte original {i + 1}:\n{textes[i]}\n")
    print("----------------------------------------------------\n")

Texte original 1:
1

----------------------------------------------------

Texte original 2:
Balzac naquit � Tours le 16 mai 1799, le jour de la f�te de saint
Honor�, dont on lui donna le nom, qui parut bien sonnant et de bon
augure. Le petit Honor� ne fut pas un enfant prodige; il n'annon�a pas
pr�matur�ment qu'il ferait la Com�die humaine. C'�tait un gar�on
frais, vermeil, bien portant, joueur, aux yeux brillants et doux, mais
que rien ne distinguait des autres, du moins � des regards peu
attentifs. A sept ans, au sortir d'un externat de Tours, on le mit au
coll�ge de Vend�me, tenu par des oratoriens, o� il passa pour un �l�ve
tr�s-m�diocre.

----------------------------------------------------

Texte original 3:
La premi�re partie de Louis Lambert contient sur ce temps de la vie de
Balzac de curieux renseignements. D�doublant sa personnalit�, il s'y
peint comme un ancien condisciple de Louis Lambert, tant�t en parlant en
son nom, et tant�t pr�tant ses propres sentiments � ce personn

### Pré-traitement des données

In [10]:
nlp = spacy.load("fr_core_news_sm")

def preprocess_text_spacy(text):
    # Supprimer les caractères spéciaux sauf les points, les virgules, les points d'exclamation et les points d'interrogation
    text = re.sub(r'[^\w\s.,?!]', '', text)

    # Tokeniser, mettre en minuscules, supprimer les stopwords et lemmatiser
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and token.is_alpha]

    # Retourner les éléments prétraités
    return ' '.join(tokens)

textes_preprocessed = [preprocess_text_spacy(text) for text in textes]

Vérifications

In [11]:
# Afficher les 10 premiers textes prétraités
for i in range(10):
    print(f"Texte original {i + 1}:\n{textes[i]}\n")
    print(f"Texte prétraité {i + 1}:\n{textes_preprocessed[i]}\n")
    print("----------------------------------------------------\n")

Texte original 1:
1

Texte prétraité 1:


----------------------------------------------------

Texte original 2:
Balzac naquit ï¿½ Tours le 16 mai 1799, le jour de la fï¿½te de saint
Honorï¿½, dont on lui donna le nom, qui parut bien sonnant et de bon
augure. Le petit Honorï¿½ ne fut pas un enfant prodige; il n'annonï¿½a pas
prï¿½maturï¿½ment qu'il ferait la Comï¿½die humaine. C'ï¿½tait un garï¿½on
frais, vermeil, bien portant, joueur, aux yeux brillants et doux, mais
que rien ne distinguait des autres, du moins ï¿½ des regards peu
attentifs. A sept ans, au sortir d'un externat de Tours, on le mit au
collï¿½ge de Vendï¿½me, tenu par des oratoriens, oï¿½ il passa pour un ï¿½lï¿½ve
trï¿½s-mï¿½diocre.

Texte prétraité 2:
balzac naître tours mai jour saint donner nom paraître bien sonnant bon augure petit être enfant prodige quil faire humain frais vermeil bien porter joueur oeil brillant dou rien distinguer regard attentif an sortir dun externat tour mettre tenir oratorien passer

------

### Entraînement des modèles

In [3]:
# Création d'un pipeline pour la transformation des données et l'entraînement du modèle
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Transformation des textes en vecteurs TF-IDF
    ('classifier', LogisticRegression(solver='liblinear'))  # Classification avec la régression logistique
])

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(textes, labels, test_size=0.2, random_state=42)

# Entraînement du modèle
pipeline.fit(X_train, y_train)

# Prédiction et évaluation du modèle
y_pred = pipeline.predict(X_test)

with open('output.txt', 'w') as f:
    print("Rapport de classification:\n", classification_report(y_test, y_pred), file=f)
    print("Matrice de confusion:\n", confusion_matrix(y_test, y_pred), file=f)

In [4]:
text = '''Il était une fois, une belle et son prince. La belle aimait beaucoup lire des livres, tandis que le prince était con comme ses pieds.'''
print(pipeline.predict([text]))
print(pipeline.predict_proba([text]))

['sand']
[[0.0614396  0.18555651 0.02391894 0.65751077 0.07157418]]


In [5]:
text = '''Elle connaissait si bien l'action des toilettes enfiévrantes du soir et des molles toilettes sensuelles du matin, du déshabillé troublant gardé pour déjeuner avec les amis intimes et qui laisse à la femme, jusqu'au milieu du jour, une sorte de saveur de son lever, l'impression matérielle et chaude du lit quitté et de la chambre parfumée!'''
